> __Purpose:__ Implement an initial privacy attack to quantify how much re-identiifcation and linkability risk exists from personalization parameters (the decoder used in the co-adaptation algorithm). Thus, implement a basic ML model to link the decoder matrices back to the corresponding subject.  

-- 7 Subjects, therefore pure guessing would be 14.28% correct on average

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import seaborn
import time
import pandas as pd
import pickle
import random
from kcs_ml_infr import *

random.seed(a=1)

# Load In Data

In [2]:
keys = ['METACPHS_S106', 'METACPHS_S107','METACPHS_S108', 'METACPHS_S109', 'METACPHS_S110', 'METACPHS_S111', 'METACPHS_S112']

with open('Data\cphs_data_block1.pickle', 'rb') as handle:
    #refs_block1, poss_block1, dec_vels_block1, int_vel_block1, emgs_block1, Ws_block1, Hs_block1, alphas_block1, pDs_block1, times_block1, conditions_block1 = pickle.load(handle)
    _, _, _, _, _, Ws_block1, _, _, _, times_block1, _ = pickle.load(handle)

with open('Data\cphs_data_block2.pickle', 'rb') as handle:
    #refs_block2, poss_block2, dec_vels_block2, int_vel_block2, emgs_block2, Ws_block2, Hs_block2, alphas_block2, pDs_block2, times_block2, conditions_block2 = pickle.load(handle)
    _, _, _, _, _, Ws_block2, _, _, _, times_block2, _ = pickle.load(handle)

In [3]:
# Decoder updates
W = Ws_block1[keys[0]][0]
W[1:,:,:].shape # 7199 time points x (decoder dimensions is 2 x 6)
dold = W[0]
update_ix = []
for ix,d in enumerate(W[1:]):
  if (np.array_equal(dold,d)==False):
    update_ix.append(ix)
    dold = d

update_ix.append(len(W) - 1) 
update_ix = np.asarray(update_ix)
update_ix = np.hstack([[0],update_ix])

print(update_ix.shape)
print(update_ix)

(19,)
[    0  1200  2402  3604  4806  6008  7210  8412  9614 10816 12018 13220
 14422 15624 16826 18028 19230 20432 20769]


I'm just making the flatten array input DF here

In [4]:
dec_flattened_df1 = pd.DataFrame(columns=["Subject", "Condition", "Update Number", "Flattened Decoder"])
dec_flattened_df2 = pd.DataFrame(columns=["Subject", "Condition", "Update Number", "Flattened Decoder"])

dec_flattened_df1.head()

,Subject,Condition,Update Number,Flattened Decoder


In [5]:
num_conds = 8

t0 = time.time()

for key in keys:
    participant_dec1 = Ws_block1[key]
    participant_dec2 = Ws_block2[key]
    
    for my_cond in range(num_conds):
        for update_number, update_idx in enumerate(update_ix):
            dec_flattened_df1.loc[len(dec_flattened_df1)] = [key, my_cond, update_number, np.ravel(participant_dec1[my_cond, update_idx, :, :])]
            dec_flattened_df2.loc[len(dec_flattened_df2)] = [key, my_cond, update_number, np.ravel(participant_dec2[my_cond, update_idx, :, :])]
        
t1 = time.time()
total = t1-t0  
print(total)

dec_flattened_df = pd.concat((dec_flattened_df1, dec_flattened_df2))

13.058960437774658


In [6]:
print(dec_flattened_df.shape)
dec_flattened_df.head()

(2128, 4)


,Subject,Condition,Update Number,Flattened Decoder
0,METACPHS_S106,0,0,"[0.002722144351611262, 0.002605931562722017, 0..."
1,METACPHS_S106,0,1,"[0.002722144351611262, 0.002605931562722017, 0..."
2,METACPHS_S106,0,2,"[-0.34157085409453486, 0.08410593293763585, -0..."
3,METACPHS_S106,0,3,"[-0.1738624752800762, 0.003705171262358347, 0...."
4,METACPHS_S106,0,4,"[0.8321061501817386, -1.3261242289666402, 0.28..."


# Classification
1. Logistic Regression
2. K-Nearest Neighbor
3. Gaussian Naive Bayes
4. Linear SVC
5. Stochastic Gradient Descent
6. Decision Tree Classifier
7. Gradient Boosting Trees

In [7]:
# Machine learning
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [8]:
# Set the number of k-folds
cv = 10

In [9]:
key_to_num = dict()
num_to_key = dict()
for idx, key in enumerate(keys):
    key_to_num[key] = idx
    num_to_key[idx] = key

In [10]:
# Result logs
my_metrics_cols = ['Algorithm', 'One Off Acc', 'CV Acc', 'K Folds']
res_df = pd.DataFrame(columns=my_metrics_cols)

In [11]:
my_models = [LogisticRegression(), KNeighborsClassifier(), GaussianNB(), LinearSVC(), SGDClassifier(), DecisionTreeClassifier(), GradientBoostingClassifier()]

# Compare Accuracies Of First and Last Decoders

In [12]:
dec_flattened_df.head()

,Subject,Condition,Update Number,Flattened Decoder
0,METACPHS_S106,0,0,"[0.002722144351611262, 0.002605931562722017, 0..."
1,METACPHS_S106,0,1,"[0.002722144351611262, 0.002605931562722017, 0..."
2,METACPHS_S106,0,2,"[-0.34157085409453486, 0.08410593293763585, -0..."
3,METACPHS_S106,0,3,"[-0.1738624752800762, 0.003705171262358347, 0...."
4,METACPHS_S106,0,4,"[0.8321061501817386, -1.3261242289666402, 0.28..."


In [23]:
flat_dec_expanded_df = pd.DataFrame()
for my_row in range(dec_flattened_df.shape[0]):
    test=pd.DataFrame(dec_flattened_df.iloc[my_row,3]).T
    flat_dec_expanded_df = pd.concat((flat_dec_expanded_df, test))

flat_dec_expanded_df.reset_index(inplace=True, drop=True)
flat_dec_expanded_df['Update Number'] = dec_flattened_df['Update Number']
flat_dec_expanded_df.head()

C:\Users\kdmen\AppData\Local\Temp\ipykernel_11416\1095649416.py:7: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  flat_dec_expanded_df['Update Number'] = dec_flattened_df['Update Number']


ValueError: cannot reindex on an axis with duplicate labels

In [21]:
# Get indexes where name column has value john
first_update_idxs = dec_flattened_df[~(dec_flattened_df['Update Number'] == 1)].index
last_update_idxs = dec_flattened_df[~(dec_flattened_df['Update Number'] == 18)].index
 
# Delete these row indexes from dataFrame
first_dec_df = flat_dec_expanded_df.drop(first_update_idxs)
first_dec_labels_df = pd.DataFrame(first_dec_df['Subject'].map(key_to_num))
first_dec_df.drop(['Subject', 'Condition', 'Update Number'], axis=1, inplace=True)

last_dec_df = flat_dec_expanded_df.drop(last_update_idxs)
last_dec_labels_df = pd.DataFrame(last_dec_df['Subject'].map(key_to_num))
last_dec_df.drop(['Subject', 'Condition', 'Update Number'], axis=1, inplace=True)

KeyError: 'Subject'

In [ ]:
first_dec_df.head()

In [ ]:
first_dec_labels_df.head()

Evaluate Accuracy when only using the first decoder

In [ ]:
X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(first_dec_df, first_dec_labels_df)
y_train = np.ravel(y_train)

print(X_train.shape)
X_train.head()

In [ ]:
X_train2 = pd.DataFrame()
X_test2 = pd.DataFrame()
for my_row in range(X_train.shape[0]):
    test=pd.DataFrame(X_train.iloc[my_row,0]).T
    X_train2 = pd.concat((X_train2, test))
    
for my_row in range(X_test.shape[0]):
    test=pd.DataFrame(X_test.iloc[my_row,0]).T
    X_test2 = pd.concat((X_test2, test))
X_train2.head()

In [ ]:
X_train = X_train2
X_test = X_test2

In [ ]:
first_dec_res_df = pd.DataFrame(columns=my_metrics_cols)

for model_num, model in enumerate(my_models):
    print(f"{model_num} of {len(my_models)}")
    first_dec_res_df = train_model(model, X_train, y_train, cv, first_dec_res_df)

In [ ]:
#test_df = pd.DataFrame(columns=['Algorithm', 'CV Acc', 'Test Acc', 'K Folds'])
#for model in my_models:
#    test_df = test_model(model, X_train, y_train, X_test, y_test, test_df, cv, verbose=True)

Evaluate Accuracy when only using the last decoder